Below is the code to make a knowledge graph, using information from the Thingiverse Scraper, like id number and description. From the description, information like its name and tags can also be scraped and are also used. Make sure that the csv path is the one for all scraped thingiverse models (there are around 980 on Github). Because this uses no AI, this is labeled the existing standard search that all other searches should be compared to in benchmarking.

### Only main function and output are shown.


In [ ]:
# def main(csv_path, k_threshold=True, k=1):
#    """
#    Main function to orchestrate the semantic network construction.
# 
#    Parameters:
#    - csv_path (str): Path to the CSV file containing model descriptions.
#    - k_threshold (bool): If True, apply dynamic thresholding using mean + k*std.
#                           If False, do not apply dynamic thresholding.
#    - k (float): The multiplier for the standard deviation in threshold calculation.
#               Relevant only if k_threshold is True.
# 
#    The function performs the following steps:
#    1. Connects to Neo4j.
#    2. Loads and preprocesses data.
#    3. Extracts key components and expands synonyms.
#    4. Performs Named Entity Recognition.
#    5. Weights primary nouns and incorporates synonyms in descriptions.
#    6. Calculates various similarity measures and combines them.
#    7. Constructs the knowledge graph in Neo4j with nodes and edges.
#    8. Applies additional optimizations like pruning and clustering.
#    """
#    print("Starting semantic network construction process.")
# 
# 
#    # Step 1: Connect to Neo4j
#    graph_db = connect_to_neo4j(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# 
# 
#    # Step 2: Load Data
#    df = load_data(csv_path)
# 
# 
#    # Step 3: Preprocess Data (incorporate tags with higher weight)
#    df = preprocess_dataframe(df)
# 
# 
#    # Step 4: Extract Key Components
#    df = extract_key_nouns_adjectives(df)
#    df, synonym_dict = expand_synonyms(df)
#    df = perform_ner(df)
# 
# 
#    # Step 5: Preprocess Descriptions to Weight Primary Nouns and Incorporate Synonyms
#    df = duplicate_primary_nouns(df, weight=2.0)
# 
# 
#    # Step 6: Text Similarity Calculation
#    tfidf_matrix, tfidf_vectorizer = compute_tfidf(df)
#    cosine_sim = compute_cosine_similarity_matrix(tfidf_matrix)
# 
# 
#    # Compute Sentence-BERT similarity using weighted_description
#    embedding_sim = compute_sentence_transformer_similarity(df)
# 
# 
#    # Since 'category' is omitted in this data, we skip it.
#    category_sim = None
# 
# 
#    # Compute the tag similarity
#    tag_sim = compute_tag_similarity(df)
# 
# 
#    # Combine all similarities into a hybrid matrix
#    final_sim = hybrid_similarity(
#        cosine_sim, embedding_sim, category_sim, tag_sim=tag_sim,
#        weight_cosine=0.2, weight_embedding=0.45,
#        weight_category=0.0, weight_tags=0.35
#    )
# 
# 
#    # Adjust similarity scores based on primary nouns
#    final_sim = adjust_similarity_with_primary_nouns(df, final_sim, boost=1.2, penalize=0.8)
# 
# 
#    # Adjust similarity scores based on entities
#    final_sim = adjust_similarity_with_entities(df, final_sim, boost=1.2, penalize=0.8)
# 
# 
#    # Normalize the final similarity matrix
#    final_sim = normalize_similarity(final_sim)
# 
# 
#    # Analyze similarity scores
#    analyze_similarity_scores(final_sim)
# 
# 
#    # Determine dynamic threshold if enabled
#    if k_threshold:
#        dynamic_threshold = determine_threshold_mean_std(final_sim, k=k)  # Can adjust k as needed
#    else:
#        dynamic_threshold = 0.0  # No thresholding; connect all relevant edges
# 
# 
#    # Visualize similarity distribution with threshold
#    visualize_similarity_distribution(
#        final_sim,
#        threshold=dynamic_threshold if k_threshold else None
#    )
# 
# 
#    # Step 7: Knowledge Graph Construction with Embeddings
#    model = SentenceTransformer('all-MiniLM-L6-v2')  # Initialize Sentence-BERT model
#    create_nodes_with_embeddings(graph_db, df, model)
# 
# 
#    # Step 8: Create Edges Based on Similarity
#    create_edges_batch(graph_db, df, final_sim, threshold=dynamic_threshold)  # Apply dynamic threshold
# 
# 
#    # Step 9: Additional Optimizations
#    graph_pruning(
#        graph_db,
#        threshold=dynamic_threshold if k_threshold else 0.0
#    )  # Prune edges below the dynamic threshold
#    node_clustering(
#        graph_db, df, final_sim, threshold=dynamic_threshold
#    )  # Cluster based on dynamic threshold
# 
# 
#    print("Knowledge graph construction complete.")
# 
# 
# if __name__ == "__main__":
#    # Set k_threshold=True to enable dynamic thresholding
#    # Set k_threshold=False to disable dynamic thresholding
#    # Adjust k as needed (e.g., k=1 for mean + 1*std)
#    main(CSV_PATH, k_threshold=True, k=2)

Below this is the code that allows the user to search for their specific model (model_search.py) using the knowledge graph constructed above. There is also filtering, selection, research, and other useful functionalities.

### Only main function and output are shown.


In [ ]:
# def main():
#     """
#     Main function to handle user queries and perform search on the knowledge graph with cascading filtering.
#     """
#     print("Initializing Semantic Search on Knowledge Graph...")
# 
#     # Connect to Neo4j
#     graph_db = connect_to_neo4j(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# 
#     # Fetch all models with embeddings
#     df_models = fetch_all_models(graph_db)
# 
#     # Extract embeddings as a 2D numpy array
#     model_embeddings = np.stack(df_models['embedding'].values)
# 
#     # Load Sentence Transformer model
#     print("Loading SentenceTransformer model...")
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     print("Model loaded successfully.")
# 
#     while True:
#         # Prompt user for input
#         user_query = input("\nEnter your search query (or type 'exit' to quit): ").strip()
#         if user_query.lower() == 'exit':
#             print("Exiting Semantic Search. Goodbye!")
#             break
#         if not user_query:
#             print("Empty query. Please enter a valid search term.")
#             continue
# 
#         # Preprocess the query with dynamic prioritization
#         preprocessed_query = preprocess_text(user_query)
#         if not preprocessed_query:
#             print("Failed to preprocess the query. Please try a different input.")
#             continue
# 
#         # Encode the query
#         query_embedding = model.encode(preprocessed_query, show_progress_bar=False)
# 
#         # Compute cosine similarity
#         print("Computing similarities...")
#         similarities = compute_cosine_similarity(query_embedding, model_embeddings)
# 
#         if similarities.size == 0:
#             print("Failed to compute similarities.")
#             continue
# 
#         # Display the top-k results
#         top_k = 10  # Number of top results to retrieve
#         top_results = display_results(df_models, similarities, top_k=top_k)
# 
#         if top_results is None or top_results.empty:
#             print("No similar models found.")
#             continue
# 
#         # Initialize the current search scope with the initial top results
#         current_scope = top_results
# 
#         while True:
#             # Prompt user to apply further filters or select a model
#             selected = prompt_filter(current_scope)
#             if selected == 'yes':
#                 # User wants to filter further
#                 # Prompt to select a model from current_scope
#                 try:
#                     selection = int(input(f"Select a model to filter by (1-{len(current_scope)}): ").strip())
#                     if 1 <= selection <= len(current_scope):
#                         selected_model = current_scope.iloc[selection - 1]
#                         selected_model_id = selected_model['id']
#                         print(f"\nSelected Model ID: {selected_model_id} - {selected_model['name'] if pd.notnull(selected_model['name']) else 'N/A'}")
#                     else:
#                         print(f"Please enter a number between 1 and {len(current_scope)}.")
#                         continue
#                 except ValueError:
#                     print("Invalid input. Please enter a valid number.")
#                     continue
# 
#                 # Fetch similar models to the selected model from the graph
#                 similar_df = fetch_similar_models_from_graph(graph_db, selected_model_id)
# 
#                 if similar_df.empty:
#                     print("No further models found based on your selection.")
#                     break  # Exit filtering loop
# 
#                 # Display the new set of similar models
#                 print(f"\nModels similar to Model ID: {selected_model_id}:")
#                 similar_top_k = 10  # Number of top similar models to display
#                 similar_top_results = display_results(similar_df, similar_df['similarity'].values, top_k=similar_top_k)
# 
#                 if similar_top_results is None or similar_top_results.empty:
#                     print("No similar models found in this step.")
#                     break  # Exit filtering loop
# 
#                 # Update the current scope to the new similar models
#                 current_scope = similar_top_results
# 
#             elif selected == 'no':
#                 # User does not want to filter further
#                 break  # Exit filtering loop
#             else:
#                 # User selected a model by rank to view its full summary
#                 selected_model_id = selected
#                 # Fetch the full details of the selected model
#                 try:
#                     query = """
#                     MATCH (m:Model {id: $selected_id})
#                     RETURN m.id AS id, m.name AS name, m.description AS description, m.category AS category, m.tags AS tags
#                     """
#                     record = graph_db.run(query, selected_id=selected_model_id).data()
#                     if not record:
#                         print(f"No details found for Model ID: {selected_model_id}.")
#                     else:
#                         model_details = record[0]
#                         print(f"\n--- Full Summary of Selected Model ---")
#                         print(f"ID: {model_details['id']}")
#                         print(f"Name: {model_details['name'] if pd.notnull(model_details['name']) else 'N/A'}")
#                         print(f"Category: {model_details['category'] if pd.notnull(model_details['category']) else 'N/A'}")
#                         print(f"Tags: {model_details['tags'] if pd.notnull(model_details['tags']) else 'N/A'}")
#                         print(f"Description: {model_details['description'] if pd.notnull(model_details['description']) else 'N/A'}")
#                         print(f"---------------------------------------\n")
#                 except Exception as e:
#                     print(f"Error fetching model details: {e}")
# 
#                 # After displaying the summary, ask if the user wants another search
#                 while True:
#                     another_search = input("Do you want to perform another search? (yes/no): ").strip().lower()
#                     if another_search in ['yes', 'y']:
#                         break  # Break to the main search loop
#                     elif another_search in ['no', 'n']:
#                         print("Exiting Semantic Search. Goodbye!")
#                         sys.exit(0)
#                     else:
#                         print("Please answer with 'yes' or 'no'.")
#                 break  # Exit filtering loop to perform another search
# 
# if __name__ == "__main__":
#     main()
